<a href="https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Data%20Connectivity%20and%20Email%20Services/Formatter_with_Agentic_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-openai python-dotenv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import Tool
from langchain.agents import create_agent
from typing import List, Tuple, Any
import sqlite3
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class SQLiteConnector:
    def __init__(self, db_path="data.db"):

        uri_path = "data.db"
        self.conn = sqlite3.connect(uri_path, uri=True, check_same_thread=False)
        self.cursor = self.conn.cursor()

    def list_tables(self):
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        return [row[0] for row in self.cursor.fetchall()]

    def describe_table(self, table_name):
        self.cursor.execute(f"PRAGMA table_info({table_name});")
        return self.cursor.fetchall()

    def get_schema_summary(self):
        summary = []
        for table in self.list_tables():
            cols = self.describe_table(table)
            formatted = f"Table: {table} Columns: {[col[1] for col in cols]}"
            summary.append(formatted)
        return "\n".join(summary)



In [ ]:
connector = SQLiteConnector()

def exec_sql(sql: str) -> Tuple[List[str], List[Tuple[Any, ...]]]:
    """executes sql to fetch data"""
    cur = connector.conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [d[0] for d in cur.description] if cur.description else []
    return cols, rows


sql_reader_tool = Tool(
    name="sql_reader",
    func=exec_sql,
    description="queries sql database to get data"
)

def generate_chart(cols,rows, chart_path: str):
    """Generate a chart from a columns and rows."""
    df = pd.DataFrame(rows, columns=cols)
    plt.figure(figsize=(10, 6))
    ax = df.plot(kind="bar", x=df.columns[0], y=df.columns[1], legend=False)
    ax.set_title("Chart Output")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.close()

chart_maker_tool = Tool(
    name="chart_maker",
    func=generate_chart,
    description="makes chart using matplot lib"
)


tools = [sql_reader_tool, chart_maker_tool]

prompt = PromptTemplate(
    input_variables=["input","tools"],
    template="""
You are an assistant create SQL Queries, and fetch data to create summary  and create a Chart with chart_path = chart_yy-mm-dd-hh-mm-ss.png based on the SQL Query result according to user's query: {input}
you have access to the following tools:
{tools}
"""
)


agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="prompt"
)

In [ ]:
user_question = input("Enter your message: ")
response = agent.invoke({"input": user_question})
print(response)